In [1]:
%autosave 0
import requests
import io
import pandas as pd
from datetime import date, timedelta

## scrap data from source
url="https://covid.ourworldindata.org/data/owid-covid-data.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))

## this dataset sometime no receive update, so i check the simple link 
verif_data = c[c['date'].isin([f"{date.today()}"])]

if len(verif_data) == 0:
    print("Not today, check other link")
    url="https://covid.ourworldindata.org/data/ecdc/full_data.csv"
    s=requests.get(url).content
    c=pd.read_csv(io.StringIO(s.decode('utf-8')))
    
    verif_data = c[c['date'].isin([f"{date.today()}"])]
    
    ## If with the second link data have not yet be updated, raise Error & try again later
    if len(verif_data) == 0:
        verif_data = c[c['date'].isin([f"{date.today()}"])]
        raise ValueError('Data have not receive Update at this moment, Try later !')
    
verif_data.head(3)

Autosave disabled


,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,aged_70_older,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy
235,AFG,Asia,Afghanistan,2020-08-22,37894.0,38.0,66.143,1385.0,0.0,3.143,...,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.50,64.83
402,ALB,Europe,Albania,2020-08-22,8119.0,152.0,143.143,240.0,2.0,3.000,...,8.643,11803.431,1.1,304.195,10.08,7.1,51.2,NaN,2.89,78.57
638,DZA,Africa,Algeria,2020-08-22,40667.0,409.0,429.000,1418.0,7.0,9.571,...,3.857,13913.839,0.5,278.364,6.73,0.7,30.4,83.741,1.90,76.88


In [2]:
## dump the file for using it in other nb
c.to_csv(f"AzureSC\\Base_Files\\full_data{date.today()}.csv", index=False)

In [3]:
## Load data from days -1 to evaluate the performances of model
data_load = pd.read_csv(f"AzureSC\\Pred\\predDf{date.today() - timedelta(days=1)}.csv", index_col=0)
data_load

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-08-21,France,232636.0,0.0,30516.0,0.0,0.0,0.0
0,2020-08-21,China,89606.0,0.0,4716.0,0.0,0.0,0.0
0,2020-08-21,Italy,256768.0,0.0,35446.0,0.0,0.0,0.0
0,2020-08-21,Spain,383902.0,0.0,28915.0,0.0,0.0,0.0
0,2020-08-21,United States,5662109.0,0.0,175594.0,0.0,0.0,0.0
0,2020-08-21,World,23145488.0,0.0,801752.0,0.0,0.0,0.0
0,2020-08-21,United Kingdom,323261.0,0.0,41454.0,0.0,0.0,0.0
0,2020-08-21,Germany,230198.0,0.0,9268.0,0.0,0.0,0.0
0,2020-08-21,Iran,355421.0,0.0,20440.0,0.0,0.0,0.0
0,2020-08-21,Turkey,255635.0,0.0,6078.0,0.0,0.0,0.0


In [4]:
def Eval(c,data_load,country):
    """ Factorise function for evaluate all models
    params : 
        c = data scrap today
        data_load = dataset from day -1
        country = country concerned by eval
    """
    df = c
    df = df[df['location'].isin([f"{country}"])]
    sle = df.iloc[-1]
    v0 = sle["date"]
    v1 = sle["total_cases"]
    v2 = sle["total_deaths"]
    
    res_tempo = data_load[data_load['country'].isin([f"{country}"])]
    rez1 = res_tempo.loc[0,"total_cases_predict"]
    rez2 = res_tempo.loc[0,"total_deaths_predict"]
    
    errorCase1 = rez1-v1
    errorCase2 = rez2-v2

    follow_df = pd.DataFrame()
    follow_df.loc[0,"date"] = v0
    follow_df.loc[0,"country"] = country
    follow_df.loc[0,"total_cases_predict"] = rez1
    follow_df.loc[0,"total_cases_real"] = v1
    follow_df.loc[0,"total_deaths_predict"] = rez2
    follow_df.loc[0,"total_deaths_real"] = v2
    follow_df.loc[0,"error_abs_cases"] = errorCase1
    follow_df.loc[0,"error_abs_deaths"] = errorCase2
    
    return follow_df

In [5]:
## call eval function and store dataframe on var's
follow_df1 = Eval(c,data_load,"France")
follow_df2 = Eval(c,data_load,"China")
follow_df3 = Eval(c,data_load,"Italy")
follow_df4 = Eval(c,data_load,"Spain")
follow_df5 = Eval(c,data_load,"United States")
follow_df6 = Eval(c,data_load,"World")
follow_df7 = Eval(c,data_load,"United Kingdom")
follow_df8 = Eval(c,data_load,"Germany")
follow_df9 = Eval(c,data_load,"Iran")
follow_df10 = Eval(c,data_load,"Turkey")
follow_df11 = Eval(c,data_load,"Brazil")

In [6]:
## concatenate all df in 1
frames = [follow_df1, follow_df2, follow_df3, follow_df4,
          follow_df5, follow_df6, follow_df7, follow_df8,
          follow_df9, follow_df10, follow_df11]
rapport = pd.concat(frames)
rapport

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-08-22,France,232636.0,234400.0,30516.0,30503.0,-1764.0,13.0
0,2020-08-22,China,89606.0,89616.0,4716.0,4711.0,-10.0,5.0
0,2020-08-22,Italy,256768.0,257065.0,35446.0,35427.0,-297.0,19.0
0,2020-08-21,Spain,383902.0,386054.0,28915.0,28838.0,-2152.0,77.0
0,2020-08-22,United States,5662109.0,5623727.0,175594.0,175406.0,38382.0,188.0
0,2020-08-22,World,23145488.0,22977399.0,801752.0,800321.0,168089.0,1431.0
0,2020-08-22,United Kingdom,323261.0,323313.0,41454.0,41405.0,-52.0,49.0
0,2020-08-22,Germany,230198.0,232082.0,9268.0,9267.0,-1884.0,1.0
0,2020-08-22,Iran,355421.0,354764.0,20440.0,20376.0,657.0,64.0
0,2020-08-22,Turkey,255635.0,255723.0,6078.0,6080.0,-88.0,-2.0


In [7]:
## dump dataset for use it in nb 03, in 03 nb he will be add to full_rapport then delete 
rapport.to_csv(f"AzureSC\\Rapport\\rap{date.today()}.csv")